## Setup
We will be using the Anthropic Claude model for this notebook.

In [3]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
%pip install -qU boto3 awscli botocore

Note: you may need to restart the kernel to use updated packages.


In [30]:
import json
import boto3
import pprint
from botocore.exceptions import ClientError
from botocore.client import Config

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
chat_client = boto3.client(service_name='bedrock-runtime')
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
#model_id = "anthropic.claude-v2" # we will be using Anthropic Claude 3 Haiku throughout the notebook
model_id = "anthropic.claude-instant-v1"


In [31]:
def llm_answer_generator(question_with_prompt, max_tokens, temp, top_k, top_p):
    """
    This function is used to invoke Amazon Bedrock using the finalized prompt that was created by the prompt_finder(question)
    function.
    :param question_with_prompt: This is the finalized prompt that includes semantically similar prompts, chat history,
    and the users question all in a proper multi-shot format.
    :return: The final answer to the users question.
    """
    # body of data with parameters that is passed into the bedrock invoke model request
    # TODO: TUNE THESE PARAMETERS AS YOU SEE FIT
    body = json.dumps({"prompt": question_with_prompt,
                       "max_tokens_to_sample": max_tokens,
                       "temperature": temp,
                       "top_k": top_k,
                       "top_p": top_p,
                       "stop_sequences": []
                       })
    contentType = 'application/json'
    # Invoking the bedrock model with your specifications
    response = chat_client.invoke_model(body=body,
                                    modelId=model_id,
                                    contentType=contentType)
    # the body of the response that was generated
    response_body = json.loads(response.get('body').read())
    # retrieving the specific completion field, where you answer will be
    answer = response_body.get('completion')
    # returning the answer as a final result, which ultimately gets returned to the end user
    return answer

## Configuration Parameters
Configuration parameters influence the way generative models generate text during inference. They are 
* Max New Token:
    Use this parameter to limit the number of new tokens generated by the model. This is a very basic way to keep the responses short. 
    
* Temperature:
    Helps control the randomness of the model output. In general, higher thh temperature, the higher is the randomness. The lower the temperature, the more predictable output.
    
* Sample Top K:
    Limits the model to choose a token randomly from only the top-K tokens with the highest probability. 
* Sample Top P:
    Samples tokens with the highest probability scores until the sum of the scores reaches the specified threshold value
    
---

## 1. Zero Shot Learning

In [32]:

messages = '''Human: who won the 2006 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,600, 0, 10, 0.5))

 Italy won the 2006 FIFA World Cup, which was held in Germany. Italy defeated France 5-3 on penalties in the final after a 1-1 draw following extra time.


## Modifying the Configuration Parameters

In [33]:

messages = '''Human: who won the 2006 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,600, 0.5, 250, 1))

 Italy won the 2006 FIFA World Cup, defeating France 5-3 on penalties in the final. The tournament was hosted by Germany.


---
## 2. One Shot Learning

In [34]:
messages = '''Human: Answer the question using the format shown in the context.
who won the 2010 FIFA world cup?
Spain won the 2010 FIFA World Cup. They defeated Netheerlands by 1-0. The tournament was held in South Africa. 
who won the 2006 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,600, 0.5, 250, 1))

 Italy won the 2006 FIFA World Cup. They defeated France by 5-3 in a penalty shootout after a 1-1 draw following extra time. The tournament was held in Germany.


---
## 3. Few Shot Learning

In [35]:

messages = '''Human: Answer the question using the format shown in the context.
who won the 2010 FIFA world cup?
Spain beats Netherlands by 1-0. South Africa hosted the tournament. 

who won the 1998 FIFA world cup?
France beats Brazil by 3-0. France hosted the tournament.

who won the 1994 FIFA world cup?
Brazil beats Italy by 3-2. United States hosted the tournament.


who won the 2006 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,1600, 0.5, 250, 1))

 Italy beats France by penalties (1-1 AET, 5-3 pens). Germany hosted the tournament.


## 3a. Few shot Using Graphics/emojis

In [36]:
messages = '''Human: Answer the question using the format shown in the context.
who won the 2010 FIFA world cup?
Spain beats Netherlands by 1-0. 🇿🇦 South Africa hosted the tournament. 🏆⚽

who won the 1998 FIFA world cup?
France beats Brazil by 3-0. 🇫🇷 France hosted the tournament. 🏆⚽

who won the 1994 FIFA world cup?
Brazil beats Italy by 3-2. 🇺🇸 United States hosted the tournament. 🏆⚽

who won the 2006 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,1600, 0.5, 250, 1))

 Italy beats France by penalties (5-3 in shootout; 1-1 after extra time). 🇩🇪 Germany hosted the tournament. 🏆⚽


### Modifying the Temperature 
By modifying the temperature to 1, the model **randomly** picks a winner for a future game.

In [37]:
messages = '''Human: Answer the question using the format shown in the context.
who won the 2010 FIFA world cup?
Spain beats Netherlands by 1-0. South Africa hosted the tournament. 

who won the 1998 FIFA world cup?
France beats Brazil by 3-0. France hosted the tournament.

who won the 1994 FIFA world cup?
Brazil beats Italy by 3-2. United States hosted the tournament.


who won the 2026 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,1600, 1, 250, 1))

 For the 2026 FIFA World Cup, the host country will be United States, Mexico, and Canada. The winner has not been determined yet as the tournament has not been played.


### Modifying the Temperature 
By modifying the temperature to 0, the model is accurate about a future game.

In [40]:

messages = '''Human: Answer the question using the format shown in the context.
who won the 2010 FIFA world cup?
Spain beats Netherlands by 1-0. South Africa hosted the tournament. 

who won the 1998 FIFA world cup?
France beats Brazil by 3-0. France hosted the tournament.

who won the 1994 FIFA world cup?
Brazil beats Italy by 3-2. United States hosted the tournament.


who won the 2030 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,1600, 0, 250, 1))

 The 2030 FIFA World Cup host and winner have not been determined yet. Bidding for the 2030 tournament is still ongoing.


 Here are the key details about the world record for crossing the English Channel entirely on foot:

- The record was set by Alison Streeter of the UK in 2020.

- She swam from Dover, England to Calais, France in a time of 12 hours and 37 minutes. 

- This broke the previous women's record of 13 hours and 48 minutes, set by Chloe McCardel of Australia in 2017.

- The English Channel, also known as the Strait of Dover, separates southern England from northern France. It is approximately 21 miles wide at its narrowest point.

- Swimming the English Channel entirely under one's own power is considered one of the ultimate open water swimming challenges due to the strong tides and currents.

- To have her swim officially recognized as a record, Streeter had to stay within sight of an official observer boat at all times during her crossing. Her route and time were strictly monitored.

So in summary, the current women's world record holder for swimming the English Channel entirely on foot is 

``` 
Note: You can tell the model what do to, if it is not confident 
```

In [19]:
messages = '''Human: Answer the question using the format shown in the context.
who won the 2010 FIFA world cup?
Spain beats Netherlands by 1-0. South Africa hosted the tournament. 

who won the 1998 FIFA world cup?
France beats Brazil by 3-0. France hosted the tournament.

who won the 1994 FIFA world cup?
Brazil beats Italy by 3-2. United States hosted the tournament.

who won the 2030 FIFA world cup? if you are not sure, just answer I don't know.
Assistant:'''
print(llm_answer_generator(messages,1600, 0, 250, 1))

 I don't know who won the 2030 FIFA World Cup, as it has not taken place yet. The 2030 FIFA World Cup will be the 24th FIFA World Cup, and the host country has not yet been selected. The tournament is still over 8 years away.


---
## 4. Chain of Thoughts
Asking the model to break down the problem into smaller steps.

In [20]:
messages = '''Human: Describe step by step.
who won the 2010 FIFA world cup?
Assistant:'''
print(llm_answer_generator(messages,1600, 0, 250, 1))

 Here are the step-by-step details on who won the 2010 FIFA World Cup:

1. The 2010 FIFA World Cup was held in South Africa from June 11 to July 11, 2010. This was the 19th FIFA World Cup tournament. 

2. 32 teams from around the world qualified to compete in the tournament after going through qualifying rounds in their continental confederations. 

3. The 32 teams were drawn into 8 groups of 4 teams each in the group stage. Teams played round-robin matches within their group. The top 2 teams from each group advanced to the knockout round of 16 teams.

4. In the knockout rounds, the teams played single-elimination matches. This continued until the final match that decided the World Cup champion.

5. In the final on July 11, 2010, Spain played the Netherlands at Soccer City Stadium in Johannesburg. 

6. The match was scoreless in regular time and went to extra time. Both teams scored in extra time, making it 1-1. 

7. Since the score was tied after extra time, the final went to penalty 

---
## Hallucination

In [44]:
messages = '''Human: Who made the world record of crossing the english channel entirely on foot?
Assistant:'''
print(llm_answer_generator(messages,1600, 1, 250, 1))

 Here are the key details regarding the world record for crossing the English Channel entirely on foot:

- The English Channel separates England and France and has a width of about 21 miles (34 km) at its narrowest point.

- In 1875, Captain Matthew Webb was the first person to successfully swim the English Channel without the use of artificial aids. He completed the swim in 21 hours and 45 minutes.

- The current men's world record for crossing the English Channel entirely on foot was set by French athlete Pascal Mazé in 1998. He completed the swim in 7 hours and 24 minutes. 

- The current women's world record is held by Australian swimmer Chloe McCardel, who completed the swim in 8 hours and 10 minutes in September 2011.

- To qualify as a full crossing of the English Channel on foot, swimmers must leave and enter England or France without external assistance such as boats, equipment, or clothing not permitted in a swimming competition. Wetsuits are also not allowed.

So in summary,

---

## Prompting Best Practices

*  Clear Instructions

* Provide any constraints

* Format of response

* For larger text, move the prompt to the end of the text

